In [3]:
# Load text and motion data
import torch
from transformers import AutoTokenizer, AutoModel
from tma.models.architectures.temos.textencoder.distillbert_actor import DistilbertActorAgnosticEncoder
from tma.models.architectures.temos.motionencoder.actor import ActorAgnosticEncoder
from collections import OrderedDict

modelpath = 'distilbert-base-uncased'

textencoder = DistilbertActorAgnosticEncoder(modelpath, num_layers=4)
motionencoder = ActorAgnosticEncoder(nfeats=126, vae = True, num_layers=4)

"""
load model here
You need to normalize the motion data with mean and std.
For motionx, they are stored in './deps/t2m/motionx/vector_623/Comp_v6_KLD01/meta/*.npy'
"""

motion = torch.randn(1, 64, 126)    # B = 1, T = , D = , need normalization
lengths = [64]
text_loc = textencoder(["a man is running"]).loc
motion_loc = motionencoder(motion, lengths).loc

print(text_loc.shape)
print(motion_loc.shape)


/home/cao/projects/HumanTOMATO/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


torch.Size([1, 256])
torch.Size([1, 256])


/home/cao/projects/HumanTOMATO/.venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


## separate text encoding

In [5]:
from sentence_transformers import SentenceTransformer

# t2m_textencoder = SentenceTransformer(
#                     "sentence-transformers/sentence-t5-xl"
#                 ).to("cuda")

text_encoder = SentenceTransformer(
                "sentence-transformers/paraphrase-MiniLM-L6-v2"
            )

In [15]:
import torch.nn.functional as f

sbert_embedding = torch.tensor(text_encoder.encode(["asshole"]))
sbert_embedding = f.normalize(sbert_embedding, dim=1)
# append to retrieval_sbert_embedding then save
print(sbert_embedding.shape)


torch.Size([1, 384])
